## Specifying the function to be optimized
This is a function optimization package, therefore the first and most important ingreedient is, of course, the function to be optimized.


**DISCLAIMER:** We know exactly how the output of the function below depends on its parameter. Obviously this is just an example, and you shouldn't expect to know it in a real scenario. However, it should be clear that you don't need to. All you need in order to use this package (and more generally, this technique) is a function f that takes a known set of parameters and outputs a real number

In [27]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1

## Getting Started
All we need to get started is to instanciate a BayesianOptimization object specifying a function to be optimized f, and its parameters with their corresponding bounds, pbounds. This is a constrained optimization technique, so you must specify the minimum and maximum values that can be probed for each parameter in order for it to work

In [28]:
from bayes_opt import BayesianOptimization

In [29]:
# Bounded region of parameter space
pbounds = {'x': (2, 4), 'y': (-3, 3)}

In [30]:
optimizer = BayesianOptimization(
    f = black_box_function,
    pbounds = pbounds,
    verbose = 2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state = 1,
)

The BayesianOptimization object will work out of the box without much tuning needed. The main method you should be aware of is maximize, which does exactly what you think it does.  

There are many parameters you can pass to maximize, nonetheless, the most important ones are:  


- n_iter: How many steps of bayesian optimization you want to perform. The more steps, the more likely to find a good maximum you are.
- init_points: How many steps of **random** exploration you want to perform. Random exploration can help by diversifying the exploration space.


In [31]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -7.135    |  2.834    |  1.322    |
|  2        | -7.78     |  2.0      | -1.186    |
|  3        | -7.11     |  2.218    | -0.7867   |
|  4        | -12.4     |  3.66     |  0.9608   |
|  5        | -6.999    |  2.23     | -0.7392   |


The best combination of parameters and target value found can be accessed via the property bo.max.


In [32]:
print(optimizer.max)

{'target': -6.999472814518675, 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}


In [33]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -7.135455292718879, 'params': {'x': 2.8340440094051482, 'y': 1.3219469606529488}}
Iteration 1: 
	{'target': -7.779531005607566, 'params': {'x': 2.0002287496346898, 'y': -1.1860045642089614}}
Iteration 2: 
	{'target': -7.109925819441113, 'params': {'x': 2.2175526295255183, 'y': -0.7867249801593896}}
Iteration 3: 
	{'target': -12.397162416009818, 'params': {'x': 3.660003815774634, 'y': 0.9608275029525108}}
Iteration 4: 
	{'target': -6.999472814518675, 'params': {'x': 2.2303920156083024, 'y': -0.7392021938893159}}


### Changing bounds
During the optimization process, you may realize the bounds chosen for some parameters are not adequate. For these situations you can invoke the method set_bounds to alter them. You can pass any combination of existing parameters and their associated new bounds.
***
최적화 절차 중에, 아마 설정한 파라미터 조합이 적절하지 않다고 느낄 때가 있다. 이럴때를 대비해서, **set_bound** 라는 방법을 호출할 수 있다. 그렇게함으로 기존에 존재하는 파라미터들과 그것들과 연관되어있는 new bound와의 어떤 조합이든 전달할 수 있다.

In [34]:
optimizer.set_bounds(new_bounds={"x": (-2, 3)})

In [35]:
optimizer.maximize(
    init_points=0,
    n_iter=5,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  6        | -2.942    |  1.98     |  0.8567   |
|  7        | -0.4597   |  1.096    |  1.508    |
|  8        |  0.5304   | -0.6807   |  1.079    |
|  9        | -5.33     | -1.526    |  3.0      |
|  10       | -5.419    | -2.0      | -0.5552   |


## Guiding the optimization
It is often the case that we have an idea of regions of the parameter space where the maximum of our function might lie. For these situations the BayesianOptimization object allows the user to specify specific points to be probed. By default these will be explored lazily (lazy=True), meaning these points will be evaluated only the next time you call maximize. This probing process happens before the gaussian process takes over.
***
우리의 함수가 언제 최대가 될지, 대충 감이 올 때가 있다. 그럴때 특정한 point를 지정할 수 있다. ```lazy = True``` 다음에 최대 값을 탐색하는 함수를 호출하면 그때 계산해준다는 것.

Parameters can be passed as dictionaries such as below:

In [36]:
optimizer.probe(
    params={"x": 0.5, "y": 0.7},
    lazy=True,
)

Or as an iterable. Beware that the order has to be alphabetical. You can use optimizer.space.keys for guidance
***
알파벳 순으로 그냥 순열으로 만들어도 된다. 이는 ```optimizer.space.keys```로 알 수 있다.

In [37]:
print(optimizer.space.keys)

['x', 'y']


In [38]:
optimizer.probe(
    params=[-0.3, 0.1],
    lazy=True,
)

In [39]:
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  11       |  0.66     |  0.5      |  0.7      |
|  12       |  0.1      | -0.3      |  0.1      |


## Saving, loading and restarting
By default you can follow the progress of your optimization by setting verbose>0 when instanciating the BayesianOptimization object. If you need more control over logging/alerting you will need to use an observer. For more information about observers checkout the advanced tour notebook. Here we will only see how to use the native JSONLogger object to save to and load progress from files.
***
logging/alerting에대해서 좀 더 통제가 필요하다면 observer의 사용이 필요할 것이다. observer에 대한 더 많은 정보는 [advanced-tour]. 

### Saving progress

In [40]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

The observer paradigm works by:


1. Instantiating an observer object.
2. Tying the observer object to a particular event fired by an optimizer.  


The BayesianOptimization object fires a number of internal events during optimization, in particular, everytime it probes the function and obtains a new parameter-target combination it will fire an Events.OPTIMIZATION_STEP event, which our logger will listen to.


**Caveat:** The logger will not look back at previously probed points.

In [41]:
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [42]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  13       | -12.48    | -1.266    | -2.446    |
|  14       | -3.854    | -1.069    | -0.9266   |
|  15       | -3.594    |  0.7709   |  3.0      |
|  16       |  0.8237   |  0.03435  |  1.418    |
|  17       |  0.972    | -0.1053   |  0.8701   |


### Loading progress
Naturally, if you stored progress you will be able to load that onto a new instance of BayesianOptimization. The easiest way to do it is by invoking the load_logs function, from the util submodule.

***


In [43]:
from bayes_opt.util import load_logs

In [44]:
new_optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"x": (-2, 2), "y": (-2, 2)},
    verbose=2,
    random_state=7,
)
print(len(new_optimizer.space))

0


In [45]:
load_logs(new_optimizer, logs=["./logs.json"]);

In [46]:
new_optimizer.maximize(
    init_points=0,
    n_iter=10,
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -3.548    | -2.0      |  1.741    |
|  2        | -3.04     |  1.913    |  0.3843   |
|  3        | -12.0     |  2.0      | -2.0      |
|  4        | -3.969    |  2.0      |  1.984    |
|  5        | -0.7817   | -1.238    |  0.5018   |
|  6        |  0.5291   |  0.6849   |  0.9573   |
|  7        |  0.2986   |  0.1241   |  0.1718   |
|  8        |  0.9544   |  0.2123   |  0.9765   |
|  9        |  0.715    | -0.4374   |  1.306    |
|  10       |  0.983    | -0.06741  |  1.112    |


## Next Steps
This tour should be enough to cover most usage scenarios of this package. If, however, you feel like you need to know more, please checkout the advanced-tour notebook. There you will be able to find other, more advanced features of this package that could be what you're looking for. Also, browse the examples folder for implementation tips and ideas.